In this notebook we run a simple system of purely repulsive spheres using a [Weeks-Chandler-Andersen potential](https://hoomd-blue.readthedocs.io/en/stable/module-md-pair.html#hoomd.md.pair.lj). We visualize the spheres using the povray backend and color them by distance to their nearest neighbor.

In [ ]:
import flowws
import gtar
from hoomd_flowws.Init import Init
from hoomd_flowws.Interaction import Interaction
from hoomd_flowws.Run import Run

import plato, plato.draw.povray as draw
import freud
import numpy as np
import IPython
import ipywidgets

In [ ]:
storage = flowws.DirectoryStorage()
stages = [
    Init(number=128),
    Interaction(
        type='lj', global_params=[('r_cut', 2**(1./6))], 
        pair_params=[('A', 'A', 'epsilon', 1), ('A', 'A', 'sigma', 1)]),
    Run(steps=1e3, integrator='langevin'),
    Run(steps=1e4, integrator='langevin', compress_to=.57, dump_period=1e3),
]

flowws.Workflow(stages, storage).run()

In [ ]:
num_frames = 0

def get_frame(frame=-1):
    global num_frames
    with gtar.GTAR('dump.sqlite', 'r') as traj:
        (posRec, boxRec), frames = traj.framesWithRecordsNamed(['position', 'box'])

        num_frames = len(frames)
        positions = traj.getRecord(posRec, frames[frame])
        box = traj.getRecord(boxRec, frames[frame])
    return positions, box

In [ ]:
def update(scene, frame=-1):
    (positions, box) = get_frame(frame)
    
    # get nearest-neighbor distance, rescaled to go from 0-1, as cval
    fbox = freud.box.Box.from_box(box)
    nn = freud.locality.NearestNeighbors(1.5, 1).compute(fbox, positions, positions)
    cval = nn.r_sq_list[:, 0].copy()
    cval -= np.min(cval)
    cval /= np.max(cval)

    colors = plato.cmap.cubehelix(.25 + .5*cval)
    
    for prim in scene:
        prim.colors = colors
        prim.positions = positions
        prim.diameters = np.ones(len(positions))

prim = draw.Spheres()
features = dict(ambient_light=.4)
scene = draw.Scene(prim, features=features, zoom=4.8)
update(scene)

target = '../../gallery/flowws_spheres_povray.png'
scene.save(target)
IPython.display.Image(filename=target)

In [ ]:
import plato.draw.vispy as interactive

live_scene = scene.convert(interactive)
live_scene.show()

@ipywidgets.interact(frame=(0, num_frames - 1))
def plot(frame=0):
    update(live_scene, frame)
    live_scene.render()